**Quantitative X-Ray Imaging: Tutorial 02**   
*Julia Herzen, Franziska Hinterdobler, Sara Savatović, Sami Wirtensohn*   
*03.05.2024*

# Projection-based approaches to material decomposition

In the lecture, you have seen different approaches to material decomposition in the projection-space. In the first part of this exercise, you will get the chance to try three of them for yourself. The second part concerns the calibration-based dual-energy material decomposition and invites you to think about further aspects. 



In [1]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import scipy.ndimage as ndi
import xraylib



# Set some global plotting properties
matplotlib.rcParams['image.cmap'] = 'gray'
matplotlib.rcParams['figure.figsize'] = (10,5)
matplotlib.rcParams['figure.dpi'] = 100
# choose matplotlib magic command
# opens figures in separate interactive windows
%matplotlib qt 

Last exercise, we calculated the attenuation directly from the refractive index in `xraylib`. This time, we will use the total attenuation coefficient $\mu_{tot}$ stored in `xraylib`, as it contains contributions of the photo-effect, Compton and Rayleigh scattering. You can also find the information here: https://www.nist.gov/pml/x-ray-mass-attenuation-coefficients
Note: As you can see in the function `get_attenuation_coefficient`, this implementation in xraylib actually returns the mass attenuation coefficient $\mu/\rho$ and the length scale is now cm.

In [2]:
def get_n(name, E, density=None):
    """
    Get compound refractive index values: beta, delta, mu.

    Parameters
    ----------
    name : string
        Compound chemical composition or compound name from the list
        xraylib.GetCompoundDataNISTList().
    E : array
        Energy or list of energies in keV.
    density : float, optional
        Density of the compound in g/cm³. If the compound is from the
        NISTList it is not required. Check compound specifications with 
        xraylib.GetCompoundDataNISTByName(name)

    Returns
    ----------
    out : dictionary
        Values in dictionary entries "name", beta", "delta", "mu".
    """
        
    if density is None:
        try:
            density = xraylib.GetCompoundDataNISTByName(name)['density']
        except:
            print('Specify density, compound not in NIST database.')
            return

    def _n(name, E, density):
        beta  = xraylib.Refractive_Index_Im(name, E+.0, density)
        delta = 1 - xraylib.Refractive_Index_Re(name, E+.0, density)
        lam = xraylib.KEV2ANGST * 1e-10 / E  # in m
        mu = beta * 4 * np.pi / lam
        return dict(name=name, beta=beta, delta=delta, mu=mu)

    if np.ndim(E)==0:
        E = [E]
    compound = np.array([_n(name, e, density)['name'] for e in E]).squeeze()
    beta = np.array([_n(name, e, density)['beta'] for e in E]).squeeze()
    delta = np.array([_n(name, e, density)['delta'] for e in E]).squeeze()
    mu = np.array([_n(name, e, density)['mu'] for e in E]).squeeze()
    out = dict(name=compound, beta=beta, delta=delta, mu=mu)

    return out

def get_attenuation_coefficient(compound, density, energy):
    energies = energy
    mues = np.empty_like(energies)
    for i, energy in enumerate(energies):
        mues[i] = xraylib.CS_Total_CP(compound, energy) # cm^2/g
    mue_rho = mues * density # 1/cm
    return mue_rho

def disc_mask(sh, ctr, R):
    """
    Returns disc mask of radius R, centered in ctr, for an image of shape sh.
    """
    i0, i1 = np.indices(sh, dtype=float) - np.array(ctr)[:, np.newaxis, np.newaxis]
    r2 = (i0/R)**2 + (i1/R)**2
    return (r2 <= 1.)

def imsh(img, p=5, vmin=None, vmax=None):
    """
    Function to show images with automatic windowing.
    """
    if not vmin: vmin=np.percentile(img, p)
    if not vmax: vmax=np.percentile(img, 100-p)
    plt.imshow(img, vmin=vmin, vmax=vmax)

def add_poisson_noise(img, I0=1):
    img_norm = I0 * 1e5 * (img.copy() - img.min()) / (img.max() - img.min())
    noisy = np.random.poisson(img_norm)
    out = noisy / (I0 * 1e5)
    return out

These functions may be useful to you for finding appropriate materials and their data:

In [3]:
xraylib.GetCompoundDataNISTList()
xraylib.GetCompoundDataNISTByName('Air, Dry (near sea level)')

{'name': 'Air, Dry (near sea level)',
 'nElements': 4,
 'density': 0.001205,
 'Elements': (6, 7, 8, 18),
 'massFractions': (0.000124, 0.755267, 0.231781, 0.012827)}

Here, we define our energy range and materials we want to use in our phantoms. Feel free to expand the list if you're interested in other materials.

In [4]:
energies = np.arange(20., 130.,10) # in keV
muscle = get_attenuation_coefficient('Muscle, Skeletal', 1.92, energy = energies) # in 1/cm!
soft = get_attenuation_coefficient('Tissue, Soft (ICRP)', 0.92, energy = energies)
water = get_attenuation_coefficient('Water, Liquid', 1.0, energy = energies )
iodine = get_attenuation_coefficient('I', 4.93, energy = energies)
contrast = 0.98 * water + 0.02*iodine
bone = get_attenuation_coefficient('Bone, Compact (ICRU)', 1.92, energy=energies)
lung = get_attenuation_coefficient('Lung (ICRP)', 1.05, energy=energies)

### 1) Simple subtraction
Use case: Digital subtraction angiography

Digital subtraction angiography is a clinical technique to better visualize vessels, for example in the heart. Two images are taken, one before and one after introduction of a contrast agent. Subtracting the images nullifies the unchanged tissue in the difference image. We will build a simplified phantom of arteries in soft tissue to visualize this effect. usually, this is performed using a laboratory source with a polychromatic spectrum. For demonstration purposes, our spectrum will be monochromatic.

Choose appropriate materials and energies for clinical imaging. We first build a slice and extend that to 3D, like in exercise 01.

In [5]:
E0 = 4 #index of energy in list -> 90keV
sh = (400,400)
phantom = np.ones(sh) * soft[E0]#np.zeros(sh)#
mask_art1 = disc_mask(sh, (200,180),8)
mask_art2 = disc_mask(sh, (80,80), 12)
mask_art3 = disc_mask(sh, (300,180), 40)
#mask_art4 = disc_mask(sh, (306, 380), 40)

phantom[mask_art1] = water[E0]
phantom[mask_art2] = water[E0]
phantom[mask_art3] = bone[E0]

plt.figure()
plt.title(r'Native arteries in surrounding material')
imsh(phantom, vmin=0.3, vmax=0.36)
plt.text(160, 160, 'Artery 1', c='y')
plt.text(60, 107, 'Artery 2', c='y')
plt.text(135, 360, 'Artery 3', c='y')
#plt.text(385, 306, 'Artery 4', c='y')
plt.axis('off')
cbar = plt.colorbar(pad=0.01)
cbar.set_label('Mass attenuation coefficient [1/cm]')
plt.show()

Now, we forward project the phantom to get the detector image. Choose appropriate windowing parameters for the image. Use the Poisson noise function defined above to generate a second projection that includes noise.

In [6]:
phantom_3d = np.stack(300*[phantom], axis=0)
px_size = 10e-4 # in cm
proj1 = np.exp(-(phantom_3d.sum(axis=1)*px_size))
proj1_noise = add_poisson_noise(proj1, I0 = 0.6)

plt.figure()
plt.subplot(211)
plt.title('Slice in the center of the volume')
imsh(phantom_3d[:,200,:], vmin=0.3, vmax=0.36)
plt.axis('off')
cbar = plt.colorbar(pad=0.01)
cbar.set_label('Mass att. coef. [1/cm]')
plt.subplot(212)
plt.title('Projection \n Image recorded by the detector')
imsh(proj1)#, vmin = 0.85, vmax = 0.9)
cbar = plt.colorbar(pad=0.01)
cbar.set_label("Transmission [I/$\mathrm{I}_0$]")
plt.axis('off')
plt.tight_layout()

Now we introduce the contrast agent and forward project the second phantom as well. Remember to generate a noisy projection as well.

In [7]:
phantom2 = np.ones(sh) * soft[E0]#np.zeros(sh) #
mask_art1 = disc_mask(sh, (200,180),8)
mask_art2 = disc_mask(sh, (80,80), 12)
mask_art3 = disc_mask(sh, (300,180), 40)
#mask_art4 = disc_mask(sh, (306, 380), 40)

phantom2[mask_art1] = contrast[E0]
phantom2[mask_art2] = contrast[E0]
phantom2[mask_art3] = bone[E0]

plt.figure()
plt.title(r'Contrast-filled arteries (iodine) in tissue (soft tissue)')
imsh(phantom2, vmin=0.3, vmax=0.44)
plt.text(160, 160, 'Artery 1', c='y')
plt.text(60, 107, 'Artery 2', c='y')
plt.text(135, 360, 'Artery 3', c='y')
#plt.text(385, 306, 'Artery 4', c='y')
plt.axis('off')
cbar = plt.colorbar(pad=0.01)
cbar.set_label('Linear attenuation coefficient [1/cm]')

In [8]:
phantom2_3d = np.stack(300*[phantom2], axis=0)
proj2 = np.exp(-(phantom2_3d.sum(axis=1)*px_size))
proj2_noise = add_poisson_noise(proj2, I0=0.6)

plt.figure()
plt.subplot(211)
plt.title('Slice in the center of the volume')
imsh(phantom2_3d[:,200,:], vmin=0.3, vmax=0.44)
cbar = plt.colorbar(pad=0.01)
cbar.set_label('Linear att. coef. [1/cm]')
plt.axis('off')
plt.subplot(212)
plt.title('Projection \n Image recorded by the detector')
imsh(proj2)#, vmin=0.85, vmax=0.9)
cbar = plt.colorbar(pad=0.01)
cbar.set_label("Intensity [I/$\mathrm{I}_0$]")
plt.axis('off')
plt.tight_layout()


Now subtract and look at the difference image. From the lecture, we know
$DSA = \log(I_{contrast}) - \log(I_{native})$. 

Calculate a difference image using the noisy projections, as well. Do the images differ?

In [10]:
proj1_noise = np.clip(proj1_noise,0.001, 1.1)
proj2_noise = np.clip(proj2_noise,0.001, 1.1)
diff = np.log(proj2) - np.log(proj1)
diff_noise = np.log(proj2_noise) - np.log(proj1_noise)

plt.figure()
plt.title(r'Subtracted image')
imsh(diff_noise)
cbar = plt.colorbar(pad=0.01)
cbar.set_label('Intensity difference')
plt.axis('off')

(-0.5, 399.5, 299.5, -0.5)

Question part:
Simple subtraction:
1) DSA usually uses a laboratory X-ray source. How does the polychromatic spectrum influence the method?
2) Is DSA quantitative?
3) What are limitations of the method?




## Weighted subtraction:

Now we will take a look at weighted subtraction. As you have seen in the lecture, it is a dual-energy approach. Take a look at the two tube spectra and calculate the mean energy and overlap for both.


We will now create a phantom from the Shepp-Logan phantom. It is usually used as a head phantom, but we will fill it with values usually found in the thorax.

In [11]:
# create phantom with bone, tissue and air components
phantom_LE = np.load('shepp_logan.npy')
phantom_HE = np.load('shepp_logan.npy')

# use the known attenuation values to replace the original values in the modified Shepp-Logan phantom
phantom_LE[phantom_LE == 1.0] = bone[np.where(energies == 60.)]
phantom_LE[phantom_LE == 0.19999999] = soft[np.where(energies == 60.)]
phantom_LE[phantom_LE == -1.4901161e-08] = bone[np.where(energies == 60.)]
phantom_LE[phantom_LE == 0.0] = 0.0001

phantom_HE[phantom_HE == 1.0] = bone[np.where(energies == 120.)]
phantom_HE[phantom_HE == 0.19999999] = soft[np.where(energies == 120.)]
phantom_HE[phantom_HE == -1.4901161e-08] = bone[np.where(energies == 120.)]
phantom_HE[phantom_HE == 0.0] = 0.0001

In [12]:
phantom_HE.shape

(200, 200, 200)

In [13]:
plt.figure()# figsize=(14, 6))
plt.subplot(121)
plt.imshow(phantom_LE[100,:,:], vmin=0, vmax = 0.5)
plt.title('Low energy phantom')
plt.axis('off')
cbar = plt.colorbar(pad = 0.03, fraction = 0.045)
cbar.set_label('Linear attenuation coefficient [1/cm]')
plt.tight_layout()
plt.subplot(122)
plt.imshow(phantom_HE[100,:,:], vmin=0, vmax = 0.5)
plt.title('High energy phantom')
cbar = plt.colorbar(pad = 0.03, fraction = 0.045)
cbar.set_label('Linear attenuation coefficient [1/cm]')
plt.axis('off')
plt.tight_layout()

Now again, we forwardproject and look at the resulting images. Create a second set of noisy projections:

In [15]:
px_size = 1e-02 #in cm
proj_LE = np.exp(-phantom_LE.sum(axis=1)*px_size)
LE_noise = add_poisson_noise(proj_LE, I0 = 0.8)
LE_noise = np.clip(LE_noise, 0.001, 1.1)
proj_HE = np.exp(-phantom_HE.sum(axis=1)*px_size)
HE_noise = add_poisson_noise(proj_HE, I0 = 0.8)
HE_noise = np.clip(HE_noise, 0.001, 1.1)
vmin = 0.2
vmax = 0.95 # choose max value for windowing larger than one to see the image borders

plt.figure()
plt.subplot(121)
plt.imshow(LE_noise, vmin=vmin, vmax=vmax)
plt.title('Low energy image')
plt.axis('off')
cbar = plt.colorbar(pad = 0.03, fraction = 0.045)
cbar.set_label("Intensity [I/$\mathrm{I}_0$]")
plt.subplot(122)
plt.imshow(HE_noise, vmin=vmin, vmax=vmax)
plt.title('High energy image')
plt.axis('off')
cbar = plt.colorbar(pad = 0.03, fraction = 0.045)
cbar.set_label("Intensity [I/$\mathrm{I}_0$]")
plt.tight_layout()

now there are different ways to determine the weights for weighted subtraction. One is logarithmic weights (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7015793/  #S6title) or estimating from the energy dependence or simply trying out different weights. Try it once with the (optimum) weight described in section 2.1 in the article and then, in a second approach, try to find some optimal weights by trial and error. Our goal here is to completely null the bone signal.

In [16]:
weight_opt = bone[np.where(energies == 120.)]/bone[np.where(energies == 60.)]
diff = np.log(proj_HE) - weight_opt[0]*np.log(proj_LE)
diff_noise = np.log(HE_noise) - weight_opt[0]*np.log(LE_noise)
print(weight_opt[0])
plt.figure()
#plt.imshow(diff)
plt.imshow(diff)
cbar = plt.colorbar(pad = 0.03, fraction = 0.045)
cbar.set_label('Weighted intensity difference')
plt.title('Weighted subtraction with {0:.3f} '.format(weight_opt[0]))
plt.axis('off')
plt.tight_layout()

0.5965854768594188


In [17]:
diff_img = []
weight_test = np.arange(0.0,1,0.1)
for weight in weight_test:
    calc = np.log(proj_HE) - weight*np.log(proj_LE)
    diff_img.append(calc)

In [21]:
i = 5
plt.figure()
plt.imshow(diff_img[i])
cbar = plt.colorbar(pad = 0.03, fraction = 0.045)
cbar.set_label('Weighted intensity difference')
plt.title('Weighted subtraction with weight {}'.format(weight_test[i]))
plt.axis('off')
plt.tight_layout()

## K-edge imaging

In [22]:
# for demonstration purposes use monochromatic source
# choose energies around the K-edge of iodine
k_energies = np.array([32.,34.])
ksoft = get_attenuation_coefficient('Tissue, Soft (ICRP)', 0.92, energy = k_energies)
kwater = get_attenuation_coefficient('Water, Liquid', 1.0, energy = k_energies )
kiodine = get_attenuation_coefficient('I', 4.93, energy = k_energies)
kbone = get_attenuation_coefficient('Bone, Compact (ICRU)', 1.92, energy=k_energies)
kair = get_attenuation_coefficient('Air, Dry (near sea level)', 0.001205, energy=k_energies)
kcontrast = 0.98 * kwater + 0.02*kiodine

plt.figure()
plt.plot(energies, soft, label = 'Soft tissue')
plt.plot(energies, water, label = 'Water')
plt.plot(energies,contrast, label = 'Iodine')
plt.plot(energies,bone, label = 'Bone')
plt.xlabel('Energy [keV]')
plt.ylabel('Mass attenuation coefficient')
plt.legend()
plt.semilogy()
plt.tight_layout()

In [23]:
# build phantom with iodine in it
# below kedge
E0 = 0
Kphantom_LE = np.ones(sh) * kair[E0]
mask_art1 = disc_mask(sh, (200,200),25)
mask_art2 = disc_mask(sh, (80,80), 12)
mask_art3 = disc_mask(sh, (300,180), 40)
#mask_art4 = disc_mask(sh, (306, 380), 40)

Kphantom_LE[mask_art1] = kcontrast[E0]
Kphantom_LE[mask_art2] = kwater[E0]
Kphantom_LE[mask_art3] = kbone[E0]
#stack to 3d
Kphantom_LE = np.stack(300*[Kphantom_LE], axis=0)
#above kedge
#
E1 = 1
Kphantom_HE = np.ones(sh) * kair[E1]
mask_art1 = disc_mask(sh, (200,200),25)
mask_art2 = disc_mask(sh, (80,80), 12)
mask_art3 = disc_mask(sh, (300,180), 40)
#mask_art4 = disc_mask(sh, (306, 380), 40)

Kphantom_HE[mask_art1] = kcontrast[E1]
Kphantom_HE[mask_art2] = kwater[E1]
Kphantom_HE[mask_art3] = kbone[E1]
#stack to 3d
Kphantom_HE = np.stack(300*[Kphantom_HE], axis=0)

plt.figure()
plt.subplot(121)
plt.imshow(Kphantom_LE[100,:,:], vmax = 2.0)
cbar = plt.colorbar(pad = 0.03, fraction = 0.045)
cbar.set_label('Mass attenuation coefficient [1/cm]')
plt.title('Low energy phantom')
plt.axis('off')
plt.subplot(122)
plt.imshow(Kphantom_HE[100,:,:], vmax = 2.0)
plt.title('High energy phantom')
cbar = plt.colorbar(pad = 0.03, fraction = 0.045)
cbar.set_label('Mass attenuation coefficient [1/cm]')
plt.axis('off')

(-0.5, 399.5, 399.5, -0.5)

In [24]:
# forward project both phantoms
px_size = 10e-04 # in cm
Kproj_LE = np.exp(- Kphantom_LE.sum(axis=1)*px_size)
Kproj_HE = np.exp(- Kphantom_HE.sum(axis=1)*px_size)
k_LE_noise = add_poisson_noise(Kproj_LE, I0=0.7)
k_HE_noise = add_poisson_noise(Kproj_HE, I0=0.7)

vmin = 0.6
vmax = 0.9

plt.figure()
plt.subplot(121)
#plt.imshow(Kproj_LE, vmin = vmin, vmax = vmax)
imsh(Kproj_LE)
plt.title('Low energy image')
plt.axis('off')
cbar = plt.colorbar(pad = 0.03, fraction = 0.035)
cbar.set_label("Intensity [I/$\mathrm{I}_0$]")
plt.tight_layout()
plt.subplot(122)
#plt.imshow(Kproj_HE, vmin=vmin, vmax=vmax)
imsh(Kproj_HE)
plt.title('High energy image')
plt.axis('off')
cbar = plt.colorbar(pad = 0.03, fraction = 0.035)
cbar.set_label("Intensity [I/$\mathrm{I}_0$]")
plt.tight_layout()

Now that we have built the phantom and produced the projections, we subtract. The general procedure is subtracting the logarithms of the intensities, as seen in the lecture:
$$
    KES = \log(I_{HE}) - \log(I_{LE})
$$
For getting rid of higher absorbing materials like bone, a weight factor has been proven useful. Assuming the majority of contribution is from the photoelectric effect, we can use its $E^{-3}$ dependence to calculate a weight factor:

$$
   KES = \log(I_{HE}) - \big(\frac{E_{low}}{E_{high}}\big)^3 \log(I_{LE}) 
$$

In [26]:
#do K-edge subtraction with weight factor 
k_weight = (k_energies[0]/k_energies[1])**3
kdiff = np.log(Kproj_HE) - k_weight*np.log(Kproj_LE)
k_HE_noise = np.clip(k_HE_noise, 0.001, 1.1)
k_LE_noise = np.clip(k_LE_noise, 0.001, 1.1)
kdiff_noise = np.log(k_HE_noise) - k_weight*np.log(k_LE_noise)

print(k_weight)
plt.figure()
imsh(kdiff_noise)#, vmin=0.02, vmax=0.16)
cbar = plt.colorbar(pad = 0.03, fraction = 0.045)
cbar.set_label('Weighted intensity difference')
plt.axis('off')

0.833706492977814


(-0.5, 399.5, 299.5, -0.5)

##### Questions
Compare the three methods you tried out, here are some guiding questions:
1) How did the method work out? 

2) In what way is the method quantitative?

3) What are the advantages of this method? How is it limited?

4) How does it differ from the other methods?


### Part 2: Calibration-based dual-energy material decomposition

Q: Which material properties and physical principles does dual-energy material decomposition exploit?

A: Energy dependence of attenuation coefficient (material-specific)

Q: In which way is the method quantitative?

A: Allows determining quantitative information like $Z_{\text{eff}}$, contrast agent concentration, electron density

Q: What additional considerations are necessary when using a polychromatic spectrum and not a monochromatic one?

A: Due to the energy dependence of the attenuation coefficient, the effective attenuation coefficient becomes an integral.

Q: Since the two-material approach has a reduced accuracy for higher $Z_{\text{eff}}$, you decide to increase the amount of basis materials to six. What challenges might you encounter?

A: Increases the calibration matrix to at least 6x6 --> very complicated experimentally and computationally expensive. 

Q: Why can't you use this material decomposition approach to determine the existence (yes/no) of one of your basis materials in the sample?

A: All materials are shown as linear combination of the basis materials, e.g. for a iodine/water basis, iron would also be shown as behaving like xx fraction iodine and xx fraction water, although containing neither.

Q: How could this method benefit in the clinical routine?

A: Dosis planning in radiation therapy, virtual native images (less dose)

Q: How does this method differ from the methods in Part 1? For ex. Weighted subtraction also produces a soft-tissue equivalent image, how is it different?

A: Based on a physical model, reduces artifacts.
